# Clasify news by topic related

References:
* https://newspaper.readthedocs.io/en/latest/


In [3]:
import polars as pl
from newspaper import Article
import nltk
nltk.download('punkt')
import newspaper


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sebacastillo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#url = 'https://www.paralelo32.com.ar/'
# url = 'https://www.diariouno.com.ar/' #  segundos consulta url artículos
url = 'https://www.infobae.com/' # 20 segundos build de source. Instanciado a las 10.30

In [5]:
source = newspaper.build(url, language='es')

In [6]:
for article in source.articles:
        print(article.url)

https://www.infobae.com/economia/2023/08/07/dolar-hoy-en-vivo-a-cuanto-abre-este-lunes-7-de-agosto-y-cual-es-el-precio-minuto-a-minuto/
https://www.infobae.com/politica/2023/08/07/macri-rodriguez-larreta-y-bullrich-haran-hoy-una-foto-de-unidad-en-el-cierre-de-campana-de-jorge-macri/
https://www.infobae.com/deportes/2023/08/07/el-mensaje-de-lionel-messi-en-las-redes-sociales-tras-su-brillante-actuacion-ante-dallas-la-reaccion-de-thiago-almada-y-un-agradecimiento-especial/
https://www.infobae.com/teleshow/2023/08/07/el-operativo-lm-la-teoria-que-indica-que-luis-miguel-es-reemplazado-por-un-doble-en-sus-recitales-en-argentina/
https://www.infobae.com/america/medio-ambiente/2023/08/07/los-oceanos-baten-records-de-temperaturas-y-afectan-a-las-especies-marinas-como-si-fueran-incendios/
https://www.infobae.com/deportes/2023/08/07/la-reaccion-de-beckham-tras-el-tiro-libre-de-messi-y-el-sentido-gesto-con-el-tata-martino-las-perlitas-del-triunfo-de-inter-miami/
https://www.infobae.com/deportes/2

In [7]:
# cantidad de artículos
source.size()

48

In [8]:
a = str(source.articles[1].url)

In [9]:
print(a)

https://www.infobae.com/politica/2023/08/07/macri-rodriguez-larreta-y-bullrich-haran-hoy-una-foto-de-unidad-en-el-cierre-de-campana-de-jorge-macri/


In [10]:
import re
urlshort = re.sub(url, '', a)
urlshort = re.sub(r'[^\w\s]', ' ', urlshort)
urlshort

'politica 2023 08 07 macri rodriguez larreta y bullrich haran hoy una foto de unidad en el cierre de campana de jorge macri '

In [11]:
# Tokenization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
txt_enc = tokenizer(urlshort)
print(f'url encoded {txt_enc}')
txt_tok = tokenizer.convert_ids_to_tokens(txt_enc.input_ids)
print(f'url tokenz {txt_tok}')
txt_dec = tokenizer.decode(txt_enc.input_ids)
print(f'url decode {txt_dec}')

url encoded {'input_ids': [101, 14955, 18291, 2050, 16798, 2509, 5511, 5718, 6097, 3089, 9172, 2474, 27032, 2050, 1061, 7087, 13149, 18820, 2078, 7570, 2100, 14477, 1042, 11439, 2139, 4895, 27893, 4372, 3449, 25022, 28849, 2139, 3409, 5162, 2139, 10853, 6097, 3089, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
url tokenz ['[CLS]', 'pol', '##itic', '##a', '202', '##3', '08', '07', 'mac', '##ri', 'rodriguez', 'la', '##rret', '##a', 'y', 'bull', '##rich', 'hara', '##n', 'ho', '##y', 'una', 'f', '##oto', 'de', 'un', '##idad', 'en', 'el', 'ci', '##erre', 'de', 'camp', '##ana', 'de', 'jorge', 'mac', '##ri', '[SEP]']
url decode [CLS] politica 2023 08 07 macri rodriguez larreta y bullrich haran hoy una foto de unidad en el cierre de campana de jorge macri [SEP]


# Similaridad horaciones

In [28]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('thenlper/gte-base')

In [32]:
#Compute embedding for both lists
rembeddings = model.encode(urlshort, convert_to_tensor=True)
vembeddings = model.encode("narcotráfico droga traficante", convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(rembeddings, vembeddings)
cosine_scores

tensor([[0.7803]])

In [ ]:
# Function to get the word embedding vector
def get_vector(word):
    try:
        return model[word]
    except:
        return None  # or a zero vector

# Tokenize and get embeddings for the urlshort and keywords
urlshort_vectors = [get_vector(word) for word in urlshort.split()]

# computarlo una sola vez y guardar keyword_vectors
# Pre-compute keyword vectors
keyword_vectors = {keyword: model[keyword] for keyword in keywords}

def get_most_similar_keyword(url, threshold=0.7):
    url_tokens = url.split()
    url_vectors = [model[token] for token in url_tokens if token in model]
    
    for keyword, keyword_vec in keyword_vectors.items():
        for url_vec in url_vectors:
            similarity = cosine_similarity([url_vec], [keyword_vec])[0][0]
            if similarity > threshold:
                return keyword
    return None

# Process URLs
results = {}
for url in urls:
    related_keyword = get_most_similar_keyword(url)
    results[url] = related_keyword

## Test extraction of keywords and headlines

In [82]:
url_narco = 'https://www.infobae.com/america/america-latina/2023/08/04/autoridades-bolivianas-aseguran-tener-acorralado-al-narco-profugo-sebastian-marset-2800-policias-trabajan-en-su-busqueda-y-captura/'
url_no_narco = source.articles[1].url
article_narco = Article(url_narco)
article_no_narco = Article(url_no_narco)

In [95]:
url_no_narco

'https://www.infobae.com/politica/2023/08/07/video-paso-a-paso-como-funciona-la-boleta-unica-electronica-que-se-usara-para-votar-en-caba/'

## Demora en descargar dos artículos

In [84]:
article_narco.download()
article_no_narco.download()

In [86]:
article_narco.parse()
article_no_narco.parse()

In [88]:
print(article_narco.title)
print(article_no_narco.title)

Autoridades bolivianas aseguran tener “acorralado” al narco Sebastián Marset: más de 2.800 policías buscan al prófugo
Video: paso a paso, cómo funciona la Boleta Única Electrónica que se usará para votar en CABA


No es relevante el título porque reproduce (+ / -) la url

## Sumario del artículo trabaja con renkeo de sentencias más importantes

- https://github.com/codelucas/newspaper/blob/master/newspaper/nlp.py

In [89]:
article_narco.nlp()
article_no_narco.nlp()


In [94]:
print(f'## Narco: {article_narco.summary}'), print(f'## No Narco: {article_no_narco.summary}')

## Narco: “Este sujeto se encuentra acorralado por los efectivos de la policía boliviana y de la fuerza especial de lucha contra el narcotráfico.
Pues esperemos que en el transcurso de las próximas horas podamos dar con este sujeto altamente peligroso”, expresó Mamani en una conferencia de prensa desde Asunción, donde ha sostenido reuniones con las autoridades de Paraguay.
Mamani dijo que “más de 2800 policías se desplazan para la búsqueda y localización de este sujeto”, quien escapó de un operativo policial el sábado pasado.
“Hemos realizado varios operativos de manera coordinada y simultánea entre todos los países de Sudamérica, lo cual nos está permitiendo desarticular este tipo de organizaciones criminales”, añadió.
El Gobierno boliviano envió este jueves a Paraguay a una comisión para intercambiar información sobre el caso del presunto narcotraficante uruguayo.
## No Narco: Una de las novedades será que habrá dos urnas y que el elector deberá ingresar dos veces al cuarto oscuro.
L

(None, None)

**Muy util: summary**

# Falsos positivos

Para el tratamiento de falsos positivos hay que:

- trabajar en el texto de artículo. Identificar tópico puede ser por 'sumarización' o 'tópicos'.
- 

** match por mención de keyword** : Ejemplo: "El precandidato a vicepresidente de Patricia Bullrich, Luis Petri visitó Formosa, en dialogo con el Grupo de Medios TVO habló de diferentes temas, entre ellos de la importancia que tiene que cada provincia pueda producir y generar empleo, dijo que Formosa podría ser una de las provincias agrícola ganadera más importante del país, al mismo tiempo que contó que uno de los pilares que tendrán en caso de que Bullrich llegue a la presidencia será el combate contra la inseguridad y el narcotráfico.Petri comenzó diciendo «es importante que todas las provincias puedan producir, puedan tener trabajo privado y vivir con sueldos de calidad, que haya inversiones, desarrollo y que cada vez sea menor la dependencia de los ciudadanos al Estado y así podamos fomentar las potencialidades en cada provincia».«Formosa"